In [ ]:
import pandas as pd
import os
import sys
from tqdm import tqdm


project_path = "/home/wangshu/rag/hier_graph_rag"
sys.path.append(os.path.abspath(project_path))
from src.evaluate.evaluate import get_accuracy_webqsp_qa

direct_res_path = "/mnt/data/wangshu/hcarag/FB15k/hc_index_8b/qa/global_5_15_10_15_direct_QA_.json"
direct_df = pd.read_csv(direct_res_path)
zero_df = pd.read_csv("/mnt/data/wangshu/hcarag/FB15k/webqa/baseline/zero-shot.csv")
entity_df_path = "/mnt/data/wangshu/hcarag/FB15k/KG/entity_df.csv"
entity_df = pd.read_csv(entity_df_path)
community_df_path = "/mnt/data/wangshu/hcarag/FB15k/hc_index_8b/community_df_index.csv"
community_df = pd.read_csv(community_df_path)

direct_df.head(2)

,question,answers,raw_result,pred,label
0,what does jamaican people speak?,"['Jamaican Creole English Language', 'Jamaican...",---Direct Answer---: English | Patois\n---Brie...,English | Patois,Jamaican Creole English Language|Jamaican English
1,what did james k polk do before he was president?,['Lawyer'],---Direct Answer---: lawyer | politician\n---B...,lawyer | politician,Lawyer


In [3]:
zero_df


,id,question,label,pred
0,0,what does jamaican people speak?,jamaican creole english language|jamaican english,Jamaican people primarily speak a dialect of E...
1,1,what did james k polk do before he was president?,lawyer,Before becoming the President of the United St...
2,2,what is the oregon ducks 2012 football schedule?,university of oregon,The Oregon Ducks' 2012 football schedule was:\...
3,3,who plays ken barlow in coronation street?,tony warren,Barry Evans (1974-1978) and then William Roach...
4,4,what happened after mr. sugihara died?,yaotsu,"You're referring to Chiune Sugihara, the Japan..."
...,...,...,...,...
2027,2027,what team did david beckham play for before la...,preston north end f.c.,"Before joining the LA Galaxy, David Beckham pl..."
2028,2028,who is the current leader of france 2010?,nicolas sarkozy,The answer to this question would be:\n\nNicol...
2029,2029,where was the palace of knossos located?,crete|greece,The Palace of Knossos is an ancient Minoan pal...
2030,2030,where is roswell area 51?,roswell,A question that sparks curiosity and conspirac...


In [2]:
entity_df_description = entity_df["node_description"].values
entity_df_description = " ".join(entity_df_description)
print(entity_df_description[0:100])

Sire Records, American record label Theodore Bikel, Austrian-American actor, folk singer, musician, 


In [3]:
community_df_summary = community_df["summary"].values
community_title = community_df['title'].values

community_content = " ".join(community_df_summary) + " ".join(community_title)
print(community_content[0:100])

The community revolves around Equation, a mathematical statement that asserts the equality of two ex


In [5]:
text_length = []
for _, row in community_df.iterrows():
    text_length.append(len(row["summary"]) + len(row["title"]))

community_mean_text_length = sum(text_length) / len(text_length)
print(community_mean_text_length)

text_length_entity = []
for _, row in entity_df.iterrows():
    text_length_entity.append(len(row["node_description"]))
entity_mean_text_length = sum(text_length_entity) / len(text_length_entity)
print(entity_mean_text_length)


282.9520782396088
58.04563815463114


In [ ]:

cnt_entity = 0
cnt_community = 0

direct_df['answers_lists'] = direct_df['answers'].apply(lambda x: eval(x))
print(type(direct_df['answers_lists']))

for _, row in tqdm(direct_df.iterrows()):
    answer = row["answers_lists"]
    for ans in answer:
        if ans in entity_df_description:
            cnt_entity += 1
            break
        
    for ans in answer:
        if ans in community_content:
            cnt_community += 1
            break
        
    
print(cnt_entity) 
print("the answer in entity:")
print(cnt_entity / len(direct_df))
print(len(direct_df))
print("the answer in community:")
print(cnt_community)
print(cnt_community / len(direct_df))


<class 'pandas.core.series.Series'>


2032it [00:03, 533.09it/s]

1080
0.531496062992126
2032
796
0.39173228346456695


In [5]:
cnt_gr = 0

direct_df['answers_lists'] = direct_df['answers'].apply(lambda x: eval(x))
gr_base_path = "/mnt/data/wangshu/hcarag/FB15k/KG/cached_desc/"

for idx, row in tqdm(direct_df.iterrows()):
    answer = row["answers_lists"]
    with open(gr_base_path + str(idx) + ".txt", "r") as f:
        gr_text = f.read()
    for ans in answer:
        if ans in gr_text:
            cnt_gr += 1
            break
    
print(cnt_gr) 
print(cnt_gr / len(direct_df))
print(len(direct_df))

0it [00:00, ?it/s]

2032it [00:01, 1668.27it/s]

515
0.2534448818897638
2032
